In [1]:
import warnings
warnings.filterwarnings('ignore')

### 앙상블(Ensemble)
- 여러개의 분류 모델을 조합해서 더 나은 성능을 내는 방법 <br><br>
- Decision Tree모델을 증가시켜 나온 RandomForest가 대표적임 <br><br>

### RandomForest 생각을 해보자 
- 전체 데이터를 가지고 decision tree를 10번돌리나 1번돌리나 100번돌리나 똑같을 것이다 <br><br>
- 그럼 왜 RandomForest를 쓸까? <br><br>
- 이렇게 안쓰니까 쓰는거다. 어떻게 쓰냐? <br><br>
- 데이터를 랜덤하게 중복을 허용해서 몇 개를 뽑아서 의사결정 나무를 돌리고 하는 것이다. <br><br>
- 이렇게 하는 걸 부트스트랩이라고 한다. <br><br>

### RandomForest
- 부트스트랩 샘플을 사용합니다. 부트스트랩 샘플링은 중복을 허용하는 샘플링 방법<br><br>
- 샘플링 후에 샘플을 다시 복구하고 다시 샘플링 하는 방법 입니다.<br><br>
- 이와 같이 진행하는 이유는 결정트리에서 과대적합을 방지 할 수 있기 때문이다. <br><br>
- 각 결정트리에서 나오는 확률의 합을 트리갯수로 나누어 결정짓는 모델 <br><br>
- 특성의 갯수를 제곱근으로 한 것을 샘플링 하는 갯수로 한다. <br><br>
    - 피타고라스 법칙 생각하면 된다는데?? 모르겠다 <br><br>

In [14]:
import pandas as pd

wine = pd.read_csv('../Data/wine.csv')
wine.head()
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alcohol  6497 non-null   float64
 1   sugar    6497 non-null   float64
 2   pH       6497 non-null   float64
 3   class    6497 non-null   float64
dtypes: float64(4)
memory usage: 203.2 KB


In [3]:
data = wine.iloc[:,:3].to_numpy()
target = wine['class'].to_numpy()

In [4]:
from sklearn.model_selection import train_test_split

train_input , test_input , train_target , test_target = \
    train_test_split(data,target,test_size=0.2,random_state=42)

In [15]:
# RandomForest 모델
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

rf = RandomForestClassifier(
    n_jobs=-1,
    random_state=42
)

scores = cross_validate(rf,train_input,train_target,return_train_score=True,n_jobs=-1)

print(np.mean(scores['train_score']))
print(np.mean(scores['test_score']))

0.9974503966084433
0.8903224254090472


In [13]:
# 주요 feature 확인
rf.fit(train_input,train_target)
print(rf.feature_importances_)

[0.23140085 0.50085363 0.26774553]


In [18]:
# 부트스트랩 결정 시 남는 샘플 (oob : out of back)도 특성으로 구분할 수 있다. 
rf = RandomForestClassifier(oob_score=True,n_jobs=-1,random_state=42)

scores = cross_validate(rf,train_input,train_target,return_train_score=True,n_jobs=-1)

print(np.mean(scores['train_score']))
print(np.mean(scores['test_score']))

rf.fit(train_input,train_target)
rf.oob_score_ # 검증 세트의 결과라고 봐도 무방하다

0.9973541965122431
0.8905151032797809


0.8934000384837406

> ### oob를 사용해도 90%의 예측이 나온다. 
> ### 따로 검증셋을 구성하지 않아도 oob로 검증세트의 역할을 대신할 수 있다.

<br>

> ### RandomForest는 기본값 만으로도 높은 성능을 발휘하므로 자주 사용되는 모델(알고리즘)중 하나이다.

---
### Extra Tree
- 기본적으로 100개의 트리를 사용 <br><br>
- 노드 분할시 특성의 제곱근의 갯수로 사용 <br><br>
- 특성의 선택을 랜덤하게 선택한다. <br><br>
    - 데이터가 아니라 특성을 선택해서 하기 때문에 빠르다. <br><br>
- 특성의 선택을 랜덤하게 하므로 속도는 랜덤포레스트보다 빠르다. <br><br>

In [19]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1,random_state=42)
scores = cross_validate(et,train_input,train_target,return_train_score=True,n_jobs=-1)

print(np.mean(scores['train_score']))
print(np.mean(scores['test_score']))

0.9974503966084433
0.8887848893166506


In [20]:
et.fit(train_input, train_target)
et.feature_importances_

array([0.20183568, 0.52242907, 0.27573525])

---
### Gradient Boosting (그레디언트 부스팅)
- 가장 유명한 알고리즘 중 하나이다. <br><br>
- 경사하강법 처럼 손실함수를 사용한다. <br><br>
- 손실함수를 보고 트리를 추가하여 최적의 값을 도출하는 방법 <br><br>
- 경사를 이동하면서 경사의 이동거리를 제어하는 learning-rate(default : 0.1)를 사용한다. <br><br>
- max depth를 3으로 제어하여 깊이가 낮으므로 과대적합 방지 <br><br>
- 단점은 손실함수를 보고 트리를 추가하면서 진행하는 모델이므로 병렬처리를 할 수 없다. <br><br>

In [22]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True,n_jobs=-1)

print(np.mean(scores['train_score']))
print(np.mean(scores['test_score']))

0.8513568274934645
0.8474144887835937


tree 추가를 500개까지 하고 learning rate를 0.2로 변경하자

In [31]:
gb = GradientBoostingClassifier(random_state=42,learning_rate=0.2,n_estimators=500)
scores = cross_validate(gb, train_input, train_target, return_train_score=True,n_jobs=-1)

print(np.mean(scores['train_score']))
print(np.mean(scores['test_score']))

0.9464595437171814
0.8780082549788999


In [32]:
gb.fit(train_input,train_target)
gb.feature_importances_

array([0.15872278, 0.68010884, 0.16116839])

---
### 히스토그램 기반 그레디언트 부스팅(Histogram gradient boosting)
- 훈련데이터를 256개 구간으로 나누어서 훈련시키는 방법.
- 특성의 범위가 제한되어 있어 빠른 속도를 제공한다.
- 제한된 구간이므로 과대적합을 방지한다.
- 아직은 실험단계인 모델이다.

In [33]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']))
print(np.mean(scores['test_score']))

0.9321723946453317
0.8801241948619236


/Users/kwonsoonhyong/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


### XGBoost
- kaggle에서 많이 사용

In [34]:
!pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 3.8 MB/s eta 0:00:00a 0:00:01


In [37]:
from xgboost import XGBClassifier

xgb = XGBClassifier(
    tree_method = 'hist',
    random_state = 42,
    use_label_encoder = False,
    eval_metrics = 'logloss' # 히스토그램 그레디언트 부스팅을 갔다가 쓰겠다는 것이다. 
)

scores = cross_validate(xgb,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']))
print(np.mean(scores['test_score']))

/Users/kwonsoonhyong/miniforge3/envs/tensorflow/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/Users/kwonsoonhyong/miniforge3/envs/tensorflow/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[18:10:01] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "eval_metrics" } are not used.

[18:10:01] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "eval_metrics" } are not used.

[18:10:01] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "eval_metrics" } are not used.

[18:10:01] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "eval_metrics" } are not used.

[18:10:01] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "eval_metrics" } are not used.

0.9555033709953124
0.8799326275264677


---
### 번외 기능
#### Permutaion Importance (치환 중요도)
- 각 Feature별 Sample을 섞어서 계산을 한 후에 원래 Sample들과의 차이를 계산해서 차이가 많이 나는 Feature가 중요하다고 판단 <br><br>
- 즉 어느 Feature가 중요한지 파악하는 방법 <br><br>
- 어떤 모델에도 사용가능하며 특정을 파악하는 주요 기준으로 사용가능하다. (권장사항) <br><br>

In [41]:
from sklearn.inspection import permutation_importance

# 컬럼들의 index를 막 섞어 놓고 중요도를 보는 것
gb.fit(train_input,train_target)
result = permutation_importance(gb,train_input,train_target,n_repeats=10,random_state=42,n_jobs=-1)
result.importances_mean

array([0.09661343, 0.24265923, 0.09120647])

### Permutation Importace 한 후에 컬럼별 중요도 판단할 수 있는 방법
- 후에 중요도의 확률이 낮아지면 그 컬럼은 중요하다고 할 수 있다. 

|1|2|
|---|---|
|원래|array([0.15872278, 0.68010884, 0.16116839])|
|permutaion importance 후|array([0.09661343, 0.24265923, 0.09120647])|


---
# 앙상블 정리
- 앙상블 학습은 정형데이터에서 가장 뛰어난 성능을 내는 머신러닝 알고리즘 중 하나 입니다. 

### 랜덤포레스트
- 부트스트랩 샘플 사용. 대표 앙상블 학습 알고리즘이다. 

### 엑스트라 트리
- 결정트리의 노드를 랜덤하게 분할함.

### gradient boosting
- 트리의 손실을 보완하는 식으로 얕은 결정트리를 연속하여 추가함.

### 히스토그램 기반 그레디언트 부스팅
- 훈련데이터를 256개 정수 구간으로 나누어 빠르고 높은 성능을 냄.

### 기타
- XGBoost